In [ ]:
from collections import Counter
from tqdm import tqdm
from scipy.sparse import csr_matrix
import math
import operator
from sklearn.preprocessing import normalize
import numpy as np 
import warnings
import re
import nltk
from nltk.corpus import stopwords
warnings.filterwarnings("ignore")
import pandas as pd
from collections import Counter
from scipy.sparse import csr_matrix
import os
from nltk.corpus import stopwords

corpus = [
    'This is Term Frequency',
    'This is Document Frequency',
    'This is Inverse Document Frequency',
    'This is Term Frequency - Inverse Document Frequency'            
]


#text preprocessing

stop = set(stopwords.words('english'))#set of stop words
sno = nltk.stem.SnowballStemmer('english')#initialising the Snowball Stemmer

def cleanhtml(sentence): #function to clean the word of html-tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    return cleantext

def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[ ? | ! | \' | " | # ] ' , r'' , sentence)
    cleaned = re.sub(r'[ . | , | ) | ( | \ | /]' , r'', cleaned)                     
    return cleaned

#code for implementing step-by-step the checks mentioned in pre-processing
i = 0
str1 = ' '
final_string =[]
s = ''
for sent in corpus:
    filtered_sentence =[]
    #print(sent);
    sent = cleanhtml(sent) #remove HTML tags
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if(cleaned_words.isalpha()):
                s = (sno.stem(cleaned_words.lower())).encode('utf8')
                filtered_sentence.append(s)
                        
            else:
                continue
    #print(filtered_sentence)
    str1 = b" ".join(filtered_sentence) #final string of clean words
    str1 =str1.lower()
    final_string.append(str1)
    i += 1


#print(type(final_string))
#print(type(corpus))
#print(final_string)
#print(corpus)
#corpus =list(final_string)
#unique_words = set() # at first we will initialize an empty set
# check if its list type or not
if isinstance(corpus, (list,)):
    for row in corpus: # for each review in the dataset
        for word in row.split(" "): # for each word in the review. #split method converts a string into list of words
            if len(word) < 2:
                continue
            unique_words.add(word)
    unique_words = sorted(list(unique_words))
#TF
def TF(corpus):
    tf_dict = {}
    corpusCount = len(corpus)
    for word, count in corpus.items():
        tf_dict[word] = count/float(corpusCount)
    return(tf_dict)

#IDF
def IDF(corpus,unique_words):
    idf_dict={}
    corpusCount=len(corpus)
    for i in unique_words:
        count=0
        for sen in corpus:
            if i in sen.split():
                count=count+1
            idf_dict[i]=1+(math.log((1+corpusCount)/(count+1))) #1+log(N/n)
            #adding 1 to avoiding divid by zero errors #SCIKIT FORMULA
    return idf_dict 

#TFDIF

def TFIDF(tfs,idfs):
    tfidf_dict={}
    for word, val in tfs.items():
        tfidf[word] = val*idfs[word]
    return(tfidf_dict)

# it accepts only list of sentances
    
#we need prepocessing text here
def fit(dataset):    
    unique_words = set() # at first we will initialize an empty set
    # check if its list type or not
    if isinstance(dataset, (list,)):
        for row in dataset: # for each review in the dataset
            for word in row.split(" "): # for each word in the review. #split method converts a string into list of words
                if len(word) < 2:
                    continue
                unique_words.add(word)
        unique_words = sorted(list(unique_words))
        vocab = {j:i for i,j in enumerate(unique_words)}
        Idf_values_of_all_unique_words = IDF(dataset,unique_words)    
    else:
        print("you need to pass list of sentance")
    return vocab


Vocabulary=fit(corpus)
idf_of_vocabulary=fit(corpus)

print('fit')
print(idf_of_vocabulary)
print('features')
print(list(Vocabulary.keys()))     
print(list(idf_of_vocabulary.values()))
print('idf')
idf = IDF(corpus,unique_words)
print(idf.values())

#we need prepocessing text here

def transform(dataset,vocabulary,idf_values):
    sparse_matrix= csr_matrix( (len(dataset), len(vocabulary)), dtype=np.float64)
    for row  in range(0,len(dataset)):
        number_of_words_in_sentence=Counter(dataset[row].split())
        for word in dataset[row].split():
            if word in  list(vocabulary.keys()):
                tf_idf_value=(number_of_words_in_sentence[word]/len(dataset[row].split()))*(idf_values[word])
                sparse_matrix[row,vocabulary[word]]=tf_idf_value
        #print("NORM FORM\n",normalize(sparse_matrix, norm='l2', axis=1, copy=True, return_norm=False))
        output =normalize(sparse_matrix, norm='l2', axis=1, copy=True, return_norm=False)
    return output

final_output=transform(corpus,Vocabulary,idf_of_vocabulary)
print('shape')
print(final_output.shape) 

## SkLearn

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)
skl_output = vectorizer.transform(corpus)
print('^^^^^^^^^^^^')
print('SkLearn')
print('features')
print(vectorizer.get_feature_names())
print('idf')
print(vectorizer.idf_)
print('shape')
skl_output.shape
        